In [ ]:
! pip install pandas numpy faker

In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

In [3]:
fake = Faker()

def generate_expenses_for_period(num_users, start_date, end_date, output_file="expenses.csv"):
    # 월고정비 항목 및 비용 범위
    monthly_fixed_costs = {
        "월세": (450000, 550000), "관리비": (80000, 120000),"전기요금": (30000, 70000),"수도요금": (20000, 40000),
        "가스요금": (20000, 40000),"인터넷비": (25000, 35000),"통신비": (40000, 60000),"보험료": (80000, 120000),
        "교통비": (40000, 60000),"자동차할부금": (250000, 350000),"학원비": (150000, 250000),"넷플릭스구독": (12000, 14000),
        "음악구독": (9000, 11000),"미용실": (20000, 150000),"적금": (180000, 220000)
    }

    # 연고정비 항목 및 비용 범위
    annual_fixed_costs = {
        "자동차보험료": (500000, 1000000),"헬스장": (450000, 550000),"카드연회비": [5000, 10000, 20000, 50000]
    }

    # 상시비용 (매일 0~2회) 항목 및 비용 범위 설정
    daily_expenses_very_high_freq = {
        "xx식당": (6000, 20000), "xx카페": (1000, 10000)
    }

    # 상시비용 (한달 5~10회) 항목 및 비용 범위 설정
    expenses_high_freq = {
        "편의점": (500, 20000)
    }

    # 상시비용 (한달 2~5회) 항목 및 비용 범위 설정
    monthly_expenses_mid_freq = {
        "주유비": (30000, 100000), "택시비": (3800, 50000),"택배비": (3000, 5000), "도서구매": (3000, 30000),
        "영화": [12000, 15000], "생활용품구매": (10000, 50000)
    }

    # 상시비용 (1년 2~5회) 항목 및 비용 범위 설정
    annual_expenses_low_freq = {
        "교육비": (80000, 120000), "병원비": (30000, 70000), "약값": (10000, 30000), "의류비": (20000, 300000),
        "여행비": (500000, 1500000), "가구구매": (300000, 700000), "공연관람": (50000, 100000), "자동차수리": (200000, 400000),
        "화장품구매": (20000, 80000), "건강식품구매": (10000, 50000), "전자제품구매": (500000, 1500000)
    }

    def generate_random_expense(expense_dict):
        """ 주어진 비용 딕셔너리에서 랜덤한 비용 항목과 금액을 반환 """
        key = random.choice(list(expense_dict.keys()))
        value = expense_dict[key]
        if isinstance(value, tuple):
            return key, random.randint(*value)
        elif isinstance(value, list):
            return key, random.choice(value)
        else:
            return key, value

    def get_last_day_of_month(year, month):
        """ 주어진 연도와 월의 마지막 날을 반환 """
        if month == 12:
            return 31
        return (datetime(year, month + 1, 1) - timedelta(days=1)).day

    expenses = []
    for i in range(num_users):
        user_id = f'user_{i+1}'
        current_date = start_date

        # 연간 고정비용(1회) 생성
        for expense in annual_fixed_costs.keys():
            expense_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
            amount = annual_fixed_costs[expense]
            if isinstance(amount, tuple):
                amount = random.randint(*amount)
            elif isinstance(amount, list):
                amount = random.choice(amount)
            expenses.append((user_id, expense_date, expense, amount))

        # 지정된 기간 동안의 소비내역 생성
        while current_date <= end_date:

            # 월별 고정비용 (매월 1일)
            if current_date.day == 1:
                for expense, amount_range in monthly_fixed_costs.items():
                    amount = random.randint(*amount_range)
                    expenses.append((user_id, current_date, expense, amount))

            # 빈도 매우 높은 상시비용 (매일 0~2회)
            for _ in range(random.randint(0, 2)):
                expense, amount = generate_random_expense(daily_expenses_very_high_freq)
                expenses.append((user_id, current_date, expense, amount))

            # 빈도 높은 상시비용 (한달에 5~10회)
            if current_date.day == 1:  # 매월 1일에 해당 월의 발생 횟수를 결정
                last_day_of_month = get_last_day_of_month(current_date.year, current_date.month)
                high_freq_days = random.sample(range(1, last_day_of_month + 1), random.randint(5, 10))  # 한달에 5~10회 발생
            if current_date.day in high_freq_days:
                for _ in range(random.randint(1, 3)):
                    expense, amount = generate_random_expense(expenses_high_freq)
                    expenses.append((user_id, current_date, expense, amount))

            # 빈도 중간 상시비용 추가 (한달에 3회 정도)
            if random.randint(1, 10) <= 3:
                for _ in range(random.randint(1, 3)):
                    expense, amount = generate_random_expense(monthly_expenses_mid_freq)
                    expenses.append((user_id, current_date, expense, amount))

            # 빈도 낮은 상시비용 추가 (한 해에 5회 정도)
            if random.randint(1, 365) <= 5:
                for _ in range(random.randint(1, 3)):
                    expense, amount = generate_random_expense(annual_expenses_low_freq)
                    expenses.append((user_id, current_date, expense, amount))

            # 날짜를 하루 증가
            current_date += timedelta(days=1)

    # 데이터프레임 생성
    df = pd.DataFrame(expenses, columns=["user_id", "date", "expense", "amount"])

    # CSV 파일로 저장
    df.to_csv(f'.\data\{output_file}', index=False, encoding='cp949')
    return df

# 생성 유저 수, 시작-끝 일정 설정
num_users = 1
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)

# 함수 호출
expense_df = generate_expenses_for_period(num_users, start_date, end_date)
print(expense_df.head())  # 데이터프레임의 첫 몇 줄을 출력

# 데이터프레임을 CSV 파일로 저장하는 함수 호출
generate_expenses_for_period(num_users, start_date, end_date, output_file="expenses.csv")

  user_id       date expense  amount
0  user_1 2023-10-08  자동차보험료  597739
1  user_1 2023-04-12     헬스장  463549
2  user_1 2023-01-10   카드연회비    5000
3  user_1 2023-01-01      월세  474190
4  user_1 2023-01-01     관리비  104672


,user_id,date,expense,amount
0,user_1,2023-06-09,자동차보험료,582439
1,user_1,2023-09-15,헬스장,484047
2,user_1,2023-05-30,카드연회비,20000
3,user_1,2023-01-01,월세,526454
4,user_1,2023-01-01,관리비,105182
...,...,...,...,...
949,user_1,2023-12-31,xx식당,9494
950,user_1,2023-12-31,xx식당,8520
951,user_1,2023-12-31,도서구매,15894
952,user_1,2023-12-31,택시비,13566
